# CCTV Cross-Camera Person Tracking and Analysis
# ============================================

This notebook implements cross-camera person tracking and re-identification using:

- YOLOX for person detection
- ByteTracker for single-camera tracking
- Deep Person ReID for cross-camera person re-identification
- InsightFace for demographic analysis

## Setup and Imports

In [3]:
import os
import sys
from pathlib import Path
import logging
from datetime import datetime
import numpy as np
import cv2
import torch
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
import wget

In [4]:
# Install required packages
!pip install yolox
!pip install torch torchvision
!pip install torchreid  # for person re-identification

  Using cached yolox-0.3.0.tar.gz (79 kB)
  Preparing metadata (setup.py) ... done
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Using cached scikit_image-0.24.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (14 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
  Using cached ninja-1.11.1.1-py2.py3-none-macosx_10_9_universal2.macosx_10_9_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl.metadata (5.3 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached pycocotools-2.0.8-cp312-cp312-macosx_10_9_universal2.whl.metadata (1.1 kB)
  Using cached onnx-1.8.1.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of yolox to determine which version is compatible with other requirements. This could take a while.
  Using cached yolox-0.2.0.ta

In [5]:
# YOLOX and ByteTracker imports
from yolox.tracker.byte_tracker import BYTETracker
from yolox.exp import get_exp

ModuleNotFoundError: No module named 'yolox'

In [6]:
# Deep Person ReID imports
import torchreid
from torchreid import models
from torchreid import utils

/Users/chenm/GitHub/cctv-analysis/venv/lib/python3.12/site-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [7]:
# Demographic analysis (using InsightFace)
from insightface.app import FaceAnalysis

ModuleNotFoundError: No module named 'insightface'

## Configuration

In [8]:
# YOLOX weights
# Download YOLOX weights (choose one based on your needs)
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_m.pth
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.pth
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.pth

zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget


In [ ]:
# Then download the weights using torchreid's model zoo
import torchreid
# OSNet weights
torchreid.utils.download_url(
    'https://drive.google.com/uc?id=1vduhq5DpN2q1g4fYEZfPI17MJeh9qyrA',
    'osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
)

In [ ]:
def download_if_not_exists(url, output_path):
    if not os.path.exists(output_path):
        print(f"Downloading {output_path}...")
        wget.download(url, output_path)
        print("\nDownload complete!")
    else:
        print(f"{output_path} already exists.")

In [ ]:
# Download YOLOX-L weights
yolox_l_url = "https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.pth"
download_if_not_exists(yolox_l_url, "models/detection/yolox_l.pth")

NameError: name 'os' is not defined

In [ ]:
# Uncomment below lines if you want to download larger models

# Download YOLOX-S weights (smallest but fastest)
# yolox_s_url = "https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth"
# download_if_not_exists(yolox_s_url, "models/detection/yolox_s.pth")

# # Download YOLOX-M weights (medium size)
# yolox_m_url = "https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_m.pth"
# download_if_not_exists(yolox_m_url, "models/detection/yolox_m.pth")

In [ ]:
class Config:
    # YOLOX Config
    YOLOX_EXP_FILE = "yolox_s"  # or yolox_m, yolox_l, yolox_x
    YOLOX_WEIGHTS = "yolox_s.pth"
    CONFIDENCE_THRESHOLD = 0.5

    # ByteTracker Config
    TRACK_BUFFER = 30
    TRACK_THRESH = 0.5

    # ReID Config
    REID_MODEL = 'osnet_x1_0'
    REID_WEIGHTS = 'osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'

    # Video Processing
    BATCH_SIZE = 4
    INPUT_SIZE = (608, 1088)  # (height, width)

    # Matching Config
    MAX_COSINE_DISTANCE = 0.3
    # maximum time difference (in seconds) for matching
    MAX_TIME_DIFFERENCE = 300